Make DataSets

1. All features + Reduced Embeddings 6 [done]
2. All features + Reduced Embeddings 8 [done]
3. All features + AllEmbeddings [done]
4. GE(10) + W2V(6) [done]
5. GE(10) + W2V(8) [done]
6. GE(10) + AllEmbeddings [done]
7. GE(100) + W2V(6) [done]
8. GE(100) + W2V(8) [done]
9. GE(100) + AllEmbeddings [done]

In [ ]:
import os
import pandas as pd
import time
import numpy as np
from matplotlib import pyplot as plt
import itertools
import numpy as np
from scipy import stats
import pylab as pl
from sklearn import svm, linear_model
from sklearn.model_selection import cross_validate
from collections import OrderedDict
from scipy import linalg
from sklearn.preprocessing import StandardScaler

In [ ]:
pickle_path="D:\\Courses\\InfoStorage\\Project\\Data\\FE"
item_stats= pd.read_pickle(pickle_path+"\\FeaturePickle\\product_stats.pkl")
queries_input= pd.read_pickle(pickle_path+"\\FeaturePickle\\queries_input.pkl")
query_item = pd.read_pickle(pickle_path+"\\FeaturePickle\\query_item.pkl")
item_graphEmbedding10 = pd.read_pickle(pickle_path+"\\item_GraphEmbeddings10.pkl")
item_graphEmbedding100 = pd.read_pickle(pickle_path+"\\item_GraphEmbeddings100.pkl")
vector_gensimProduct6 = pd.read_pickle(pickle_path+"\\Vector_gensim_product_6.pkl")
vector_gensimProduct8 = pd.read_pickle(pickle_path+"\\Vector_gensim_product_8.pkl")
vector_gensimQuery6 = pd.read_pickle(pickle_path+"\\Vector_gensim_query_6.pkl")
vector_gensimQuery8 = pd.read_pickle(pickle_path+"\\Vector_gensim_query_8.pkl")

In [ ]:
item_stats.fillna(value=0, inplace=True)
X = pd.merge(query_item,item_stats,on='itemId',how='left')
X.head()

# GraphEmbedding10 + W2V(6)

In [ ]:
item_graphEmbedding100.head()
c=[]
for i in range(0,16):
    c.append("graphfeature"+str(i))
c.append("itemId")
item_graphEmbedding100.columns=c
item_graphEmbedding100.head()

In [ ]:
path_to_save="D:\\Courses\\InfoStorage\\Project\\Data\\Numpy_Objects\\GraphFeatures100WithWordEmbeddingsALL\\"
basicfeatures_cosine= pd.read_pickle(path_to_save+"basicfeatures_cosine.pkl")
basicfeatures_cosine.head()

In [ ]:
X = X.drop([ \
           'item_show_count','Item_clicks','Item_view_count','Item_purchase_count','userShow','userView',\
            'userPurchase','CTR','View Rate','Click Value Rate'],axis=1)
X.head()

In [ ]:
basicfeatures_cosine = basicfeatures_cosine.drop([ \
           'item_show_count','Item_clicks','Item_view_count','Item_purchase_count','userShow','userView',\
            'userPurchase','CTR','View Rate','Click Value Rate','Relevance','is.test'],axis=1)
basicfeatures_cosine.head()

In [ ]:
X = pd.merge(X,basicfeatures_cosine,on=['queryId','itemId'],how='left')

In [ ]:
X.head()

In [ ]:
len(X['vector_gensim_query'][0])

In [ ]:
X = pd.merge(X,item_graphEmbedding100,on='itemId',how='left')
X.head()

In [ ]:

#X = pd.merge(X,vector_gensimProduct6,on='itemId',how='left')
#X = pd.merge(X,vector_gensimQuery6,on='queryId',how='left')
#X = pd.merge(X,vector_gensimProduct8,on='itemId',how='left')
#X = pd.merge(X,vector_gensimQuery8,on='queryId',how='left')

Train = X[X['is.test']==False]
Test = X[X['is.test']== True]
Train= Train.drop(['is.test'], axis=1)
Test= Test.drop(['is.test'], axis=1)

In [ ]:
Train['vector_gensim_product'][0]

In [ ]:
Train.head()

In [ ]:
test=[]
query_item_order_test=[]
labels_test=[]
for index,row in Test.iterrows():
    r=[]
    query_item_order_test.append([row['queryId'],row['itemId']])
    
    r.append(row['cosine'])
    r.append(row['graphfeature0'])
    r.append(row['graphfeature1'])
    r.append(row['graphfeature2'])
    r.append(row['graphfeature3'])
    r.append(row['graphfeature4'])
    r.append(row['graphfeature5'])
    r.append(row['graphfeature6'])
    r.append(row['graphfeature7'])
    r.append(row['graphfeature8'])
    r.append(row['graphfeature9'])
    r.append(row['graphfeature10'])
    r.append(row['graphfeature11'])
    r.append(row['graphfeature12'])
    r.append(row['graphfeature13'])
    r.append(row['graphfeature14'])
    r.append(row['graphfeature15'])
    d1= row['vector_gensim_query'].tolist()
    r= r + d1[:49]
    d2= row['vector_gensim_product'].tolist()
    r= r + d2[:49]
    
    test.append(r)
    labels_test.append(row['Relevance'])

In [ ]:
train=[]
query_item_order=[]
labels=[]
for index,row in Train.iterrows():
    r=[]
    query_item_order.append([row['queryId'],row['itemId']])
    
    r.append(row['cosine'])
    r.append(row['graphfeature0'])
    r.append(row['graphfeature1'])
    r.append(row['graphfeature2'])
    r.append(row['graphfeature3'])
    r.append(row['graphfeature4'])
    r.append(row['graphfeature5'])
    r.append(row['graphfeature6'])
    r.append(row['graphfeature7'])
    r.append(row['graphfeature8'])
    r.append(row['graphfeature9'])
    r.append(row['graphfeature10'])
    r.append(row['graphfeature11'])
    r.append(row['graphfeature12'])
    r.append(row['graphfeature13'])
    r.append(row['graphfeature14'])
    r.append(row['graphfeature15'])

    d1= row['vector_gensim_query'].tolist()
    r= r + d1[:49]
    d2= row['vector_gensim_product'].tolist()
    r= r + d2[:49]
    
    train.append(r)
    labels.append(row['Relevance'])

In [ ]:
train = np.array(train)
test= np.array(test)
sc = StandardScaler()
train = sc.fit_transform(train)
test = sc.transform(test)
L=Train['queryId'].value_counts(sort=True, ascending=True)
L = dict(L)
L= OrderedDict(sorted(L.items()))
print(len(L))
print(len(labels))

In [ ]:
train.shape

In [ ]:
#http://fa.bianp.net/blog/2012/learning-to-rank-with-scikit-learn-the-pairwise-transform/
k = 0
Xp, yp, diff = [], [], []
s=0
e=0
theta = np.deg2rad(60)
w = np.array([np.sin(theta), np.cos(theta)]) #for plotting
for key in L:
    comb = itertools.combinations(range(L.get(key)), 2)
    #create the first batch
    s=e
    e=s+L.get(key) -1
    #print("Start:%d, End %d" %(s,e))
    for (i, j) in comb:
        i=i+s
        j=j+e-1
        if labels[i] == labels[j] or query_item_order[i][0] != query_item_order[j][0] :
            # skip if same target or different group(different qIds)
            continue
        Xp.append(train[i] - train[j])
        diff.append(labels[i] - labels[j])
        yp.append(np.sign(diff[-1]))
        # output balanced classes
        if yp[-1] != (-1) ** k:
            yp[-1] *= -1
            Xp[-1] *= -1
            diff[-1] *= -1
        k += 1
Xp, yp, diff = map(np.asanyarray, (Xp, yp, diff))

In [ ]:
Xp.shape

In [ ]:
clf = svm.SVC(kernel='linear', C=.1)
clf.fit(Xp, yp)
coef = clf.coef_.ravel() / linalg.norm(clf.coef_)

In [ ]:
#perform the prediction 
pred= np.dot(test, coef)

In [ ]:
# Process before Writing to a file
print(pred.shape)
print(len(pred))
od=Test['queryId'].value_counts(sort=True, ascending=True)
od = dict(od)
od= OrderedDict(sorted(od.items()))

In [ ]:
#Calculate the Scores and Get the order

q_i={}

s=0
e=0
for key in od:
    s=e
    e=s+od.get(key)
    print("Start:%d, End %d" %(s,e))
    subset= query_item_order_test[s:e]
    query_item_order=[]
    query_item_score=[]
    for item in subset:
        query_item_order.append(item[1])
        query_item_score.append(pred[s])
        s=s+1
    temp=np.argsort(query_item_score)[::-1]
    temp=temp.tolist()
    query_item_order= np.array(query_item_order)
    query_item_order = query_item_order[temp]
    q_i[key]= query_item_order.tolist()

In [ ]:
# Write to a file 
f= open("D:\\Courses\\InfoStorage\\Project\\Data\\submissions\\RankSVMGraphFeatures100WithEmbeddingsallCosineBasic.txt","w")
for key in q_i:
    f.write("%s " % key)
    for index,item in enumerate(q_i.get(key)):
        ll=len(q_i.get(key))-1
        item= int(item)
        if index < ll:
            f.write("%s," % item)
        else:
            f.write("%s" % item)
    f.write("\n")
f.close()

In [ ]:
qio = np.array(query_item_order)
lbls = np.array(labels)
qio_test = np.array(query_item_order_test)
lbls_test = np.array(labels_test)
# np.loadtxt np.savetxt
path_to_save="D:\\Courses\\InfoStorage\\Project\\Data\\Numpy_Objects\\"
X.to_pickle(path_to_save+"AllFeaturesCombined.pkl")
Train.to_pickle(path_to_save+"Train.pkl")
Test.to_pickle(path_to_save+"Test.pkl")
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddings8_train.txt',train,fmt='%1.15f')
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddingsAfterPairWise8_Xp.txt',Xp,fmt='%1.15f')
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddingsAfterPairWise8_yp.txt',yp,fmt='%1.15f')
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddings8_test.txt',test,fmt='%1.15f')
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddings8_query_item_order.txt',qio,fmt='%1.15f')
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddings8_labels.txt',lbls,fmt='%1.15f')
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddings8_query_item_order_test.txt',qio_test,fmt='%1.15f')
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddings8_labels_test.txt',lbls_test,fmt='%1.15f')

# All Features + All Embeddings

In [ ]:
Train = X[X['is.test']==False]
Test = X[X['is.test']== True]
Train= Train.drop(['is.test'], axis=1)
Test= Test.drop(['is.test'], axis=1)

In [ ]:
test=[]
query_item_order_test=[]
labels_test=[]
for index,row in Test.iterrows():
    r=[]
    query_item_order_test.append([row['queryId'],row['itemId']])
    
    r.append(row['item_show_count'])
    r.append(row['Item_clicks'])
    r.append(row['Item_view_count'])
    r.append(row['Item_purchase_count'])
    r.append(row['CTR'])
    r.append(row['userShow'])
    r.append(row['userView'])
    r.append(row['userPurchase'])
    r.append(row['View Rate'])
    r.append(row['Click Value Rate'])
    r= r + row['vector_gensim_query'].tolist()
    r= r + row['vector_gensim_product'].tolist()
    
    test.append(r)
    labels_test.append(row['Relevance'])

In [ ]:
train=[]
query_item_order=[]
labels=[]
for index,row in Train.iterrows():
    r=[]
    query_item_order.append([row['queryId'],row['itemId']])
    
    r.append(row['item_show_count'])
    r.append(row['Item_clicks'])
    r.append(row['Item_view_count'])
    r.append(row['Item_purchase_count'])
    r.append(row['CTR'])
    r.append(row['userShow'])
    r.append(row['userView'])
    r.append(row['userPurchase'])
    r.append(row['View Rate'])
    r.append(row['Click Value Rate'])
    r= r + row['vector_gensim_query'].tolist()
    r= r + row['vector_gensim_product'].tolist()
    
    train.append(r)
    labels.append(row['Relevance'])

In [ ]:
train = np.array(train)
test= np.array(test)
sc = StandardScaler()
train = sc.fit_transform(train)
test = sc.transform(test)
L=Train['queryId'].value_counts(sort=True, ascending=True)
L = dict(L)
L= OrderedDict(sorted(L.items()))
print(len(L))
print(len(labels))

In [ ]:
train.shape

In [ ]:
#http://fa.bianp.net/blog/2012/learning-to-rank-with-scikit-learn-the-pairwise-transform/
k = 0
Xp, yp, diff = [], [], []
s=0
e=0
theta = np.deg2rad(60)
w = np.array([np.sin(theta), np.cos(theta)]) #for plotting
for key in L:
    comb = itertools.combinations(range(L.get(key)), 2)
    #create the first batch
    s=e
    e=s+L.get(key) -1
    #print("Start:%d, End %d" %(s,e))
    for (i, j) in comb:
        i=i+s
        j=j+e-1
        if labels[i] == labels[j] or query_item_order[i][0] != query_item_order[j][0] :
            # skip if same target or different group(different qIds)
            continue
        Xp.append(train[i] - train[j])
        diff.append(labels[i] - labels[j])
        yp.append(np.sign(diff[-1]))
        # output balanced classes
        if yp[-1] != (-1) ** k:
            yp[-1] *= -1
            Xp[-1] *= -1
            diff[-1] *= -1
        k += 1
Xp, yp, diff = map(np.asanyarray, (Xp, yp, diff))

In [ ]:
qio = np.array(query_item_order)
lbls = np.array(labels)
qio_test = np.array(query_item_order_test)
lbls_test = np.array(labels_test)
# np.loadtxt np.savetxt
path_to_save="D:\\Courses\\InfoStorage\\Project\\Data\\Numpy_Objects\\"
X.to_pickle(path_to_save+"AllFeaturesCombined.pkl")
Train.to_pickle(path_to_save+"Train.pkl")
Test.to_pickle(path_to_save+"Test.pkl")
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_train.txt',train,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsAfterPairWiseall_Xp.txt',Xp,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsAfterPairWiseall_yp.txt',yp,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_test.txt',test,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_query_item_order.txt',qio,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_labels.txt',lbls,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_query_item_order_test.txt',qio_test,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_labels_test.txt',lbls_test,fmt='%1.15f')

# All Features + Reduced Embeddings 8

In [ ]:
vector_gensimProduct8.head()

In [ ]:
vector_gensimQuery8.head()

In [ ]:
X = X.drop(['vector_gensim_query','vector_gensim_product'],axis=1)
X.head()

In [ ]:
X = pd.merge(X,vector_gensimProduct6,on='itemId',how='left')
X = pd.merge(X,vector_gensimQuery6,on='queryId',how='left')
X.head()
Train = X[X['is.test']==False]
Test = X[X['is.test']== True]
Train= Train.drop(['is.test'], axis=1)
Test= Test.drop(['is.test'], axis=1)

In [ ]:
test=[]
query_item_order_test=[]
labels_test=[]
for index,row in Test.iterrows():
    r=[]
    query_item_order_test.append([row['queryId'],row['itemId']])
    
    r.append(row['item_show_count'])
    r.append(row['Item_clicks'])
    r.append(row['Item_view_count'])
    r.append(row['Item_purchase_count'])
    r.append(row['CTR'])
    r.append(row['userShow'])
    r.append(row['userView'])
    r.append(row['userPurchase'])
    r.append(row['View Rate'])
    r.append(row['Click Value Rate'])
    r= r + row['vector_gensim_query'].tolist()
    r= r + row['vector_gensim_product'].tolist()
    
    test.append(r)
    labels_test.append(row['Relevance'])

In [ ]:
train=[]
query_item_order=[]
labels=[]
for index,row in Train.iterrows():
    r=[]
    query_item_order.append([row['queryId'],row['itemId']])
    
    r.append(row['item_show_count'])
    r.append(row['Item_clicks'])
    r.append(row['Item_view_count'])
    r.append(row['Item_purchase_count'])
    r.append(row['CTR'])
    r.append(row['userShow'])
    r.append(row['userView'])
    r.append(row['userPurchase'])
    r.append(row['View Rate'])
    r.append(row['Click Value Rate'])
    r= r + row['vector_gensim_query'].tolist()
    r= r + row['vector_gensim_product'].tolist()
    
    train.append(r)
    labels.append(row['Relevance'])

In [ ]:
train = np.array(train)
test= np.array(test)
sc = StandardScaler()
train = sc.fit_transform(train)
test = sc.transform(test)
L=Train['queryId'].value_counts(sort=True, ascending=True)
L = dict(L)
L= OrderedDict(sorted(L.items()))
print(len(L))
print(len(labels))

In [ ]:
#http://fa.bianp.net/blog/2012/learning-to-rank-with-scikit-learn-the-pairwise-transform/
k = 0
Xp, yp, diff = [], [], []
s=0
e=0
theta = np.deg2rad(60)
w = np.array([np.sin(theta), np.cos(theta)]) #for plotting
for key in L:
    comb = itertools.combinations(range(L.get(key)), 2)
    #create the first batch
    s=e
    e=s+L.get(key) -1
    print("Start:%d, End %d" %(s,e))
    for (i, j) in comb:
        i=i+s
        j=j+e-1
        if labels[i] == labels[j] or query_item_order[i][0] != query_item_order[j][0] :
            # skip if same target or different group(different qIds)
            continue
        Xp.append(train[i] - train[j])
        diff.append(labels[i] - labels[j])
        yp.append(np.sign(diff[-1]))
        # output balanced classes
        if yp[-1] != (-1) ** k:
            yp[-1] *= -1
            Xp[-1] *= -1
            diff[-1] *= -1
        k += 1
Xp, yp, diff = map(np.asanyarray, (Xp, yp, diff))

In [ ]:
qio = np.array(query_item_order)
lbls = np.array(labels)
qio_test = np.array(query_item_order_test)
lbls_test = np.array(labels_test)
# np.loadtxt np.savetxt
path_to_save="D:\\Courses\\InfoStorage\\Project\\Data\\Numpy_Objects\\"
X.to_pickle(path_to_save+"AllFeaturesCombined.pkl")
Train.to_pickle(path_to_save+"Train.pkl")
Test.to_pickle(path_to_save+"Test.pkl")
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8_train.txt',train,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8AfterPairWise_Xp.txt',Xp,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8AfterPairWise_yp.txt',yp,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8_test.txt',test,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8_query_item_order.txt',qio,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8_labels.txt',lbls,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8_query_item_order_test.txt',qio_test,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8_labels_test.txt',lbls_test,fmt='%1.15f')

# All Features + Reduced Embeddings 6 

In [ ]:
vector_gensimProduct6.head()

In [ ]:
vector_gensimQuery6.head()

In [ ]:
X = X.drop(['vector_gensim_query','vector_gensim_product'],axis=1)
X.head()

In [ ]:
X = pd.merge(X,vector_gensimProduct6,on='itemId',how='left')
X = pd.merge(X,vector_gensimQuery6,on='queryId',how='left')

In [ ]:
X.head()

In [ ]:
Train = X[X['is.test']==False]
Test = X[X['is.test']== True]
Train= Train.drop(['is.test'], axis=1)
Test= Test.drop(['is.test'], axis=1)

In [ ]:
test=[]
query_item_order_test=[]
labels_test=[]
for index,row in Test.iterrows():
    r=[]
    query_item_order_test.append([row['queryId'],row['itemId']])
    
    r.append(row['item_show_count'])
    r.append(row['Item_clicks'])
    r.append(row['Item_view_count'])
    r.append(row['Item_purchase_count'])
    r.append(row['CTR'])
    r.append(row['userShow'])
    r.append(row['userView'])
    r.append(row['userPurchase'])
    r.append(row['View Rate'])
    r.append(row['Click Value Rate'])
    r= r + row['vector_gensim_query'].tolist()
    r= r + row['vector_gensim_product'].tolist()
    
    test.append(r)
    labels_test.append(row['Relevance'])

In [ ]:
train=[]
query_item_order=[]
labels=[]
for index,row in Train.iterrows():
    r=[]
    query_item_order.append([row['queryId'],row['itemId']])
    
    r.append(row['item_show_count'])
    r.append(row['Item_clicks'])
    r.append(row['Item_view_count'])
    r.append(row['Item_purchase_count'])
    r.append(row['CTR'])
    r.append(row['userShow'])
    r.append(row['userView'])
    r.append(row['userPurchase'])
    r.append(row['View Rate'])
    r.append(row['Click Value Rate'])
    r= r + row['vector_gensim_query'].tolist()
    r= r + row['vector_gensim_product'].tolist()
    
    train.append(r)
    labels.append(row['Relevance'])

In [ ]:
train = np.array(train)
test= np.array(test)
sc = StandardScaler()
train = sc.fit_transform(train)
test = sc.transform(test)
L=Train['queryId'].value_counts(sort=True, ascending=True)
L = dict(L)
L= OrderedDict(sorted(L.items()))
print(len(L))
print(len(labels))

In [ ]:
#http://fa.bianp.net/blog/2012/learning-to-rank-with-scikit-learn-the-pairwise-transform/
k = 0
Xp, yp, diff = [], [], []
s=0
e=0
theta = np.deg2rad(60)
w = np.array([np.sin(theta), np.cos(theta)]) #for plotting
for key in L:
    comb = itertools.combinations(range(L.get(key)), 2)
    #create the first batch
    s=e
    e=s+L.get(key) -1
    print("Start:%d, End %d" %(s,e))
    for (i, j) in comb:
        i=i+s
        j=j+e-1
        if labels[i] == labels[j] or query_item_order[i][0] != query_item_order[j][0] :
            # skip if same target or different group(different qIds)
            continue
        Xp.append(train[i] - train[j])
        diff.append(labels[i] - labels[j])
        yp.append(np.sign(diff[-1]))
        # output balanced classes
        if yp[-1] != (-1) ** k:
            yp[-1] *= -1
            Xp[-1] *= -1
            diff[-1] *= -1
        k += 1
Xp, yp, diff = map(np.asanyarray, (Xp, yp, diff))

In [ ]:
qio = np.array(query_item_order)
lbls = np.array(labels)
qio_test = np.array(query_item_order_test)
lbls_test = np.array(labels_test)
# np.loadtxt np.savetxt
path_to_save="D:\\Courses\\InfoStorage\\Project\\Data\\Numpy_Objects\\"
X.to_pickle(path_to_save+"AllFeaturesCombined.pkl")
Train.to_pickle(path_to_save+"Train.pkl")
Test.to_pickle(path_to_save+"Test.pkl")
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6_train.txt',train,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6AfterPairWise_Xp.txt',Xp,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6AfterPairWise_yp.txt',yp,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6_test.txt',test,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6_query_item_order.txt',qio,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6_labels.txt',lbls,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6_query_item_order_test.txt',qio_test,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6_labels_test.txt',lbls_test,fmt='%1.15f')